<a href="https://colab.research.google.com/github/tlsgptj/MLP/blob/MLP4/%EC%97%AC%EC%A3%BC%EC%8B%9C_%EC%97%AC%EC%A3%BC%EB%8C%80%EA%B5%90(MLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/04. 여주시_여주대교.csv", encoding="cp949")

In [ ]:
df.head(20)

In [ ]:
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = df.drop(columns=["측정소명"])

In [ ]:
plt.scatter(df["평균 상대습도(%)"], df["유량(㎥/s)"])
plt.xlabel("평균 상대습도(%)")
plt.ylabel("유량(㎥/s)")
plt.title("습도, 유량 그래프")
plt.show()

In [ ]:
#유량 및 평균상대습도 정규화
df["평균 상대습도(%)"] = df["평균 상대습도(%)"].astype(float)/abs(df["평균 상대습도(%)"].astype(float).max())
df["평균 상대습도(%)"].head()

In [ ]:
df["유량(㎥/s)"] = df["유량(㎥/s)"].astype(float)/abs(df["유량(㎥/s)"].astype(float).max())
df["유량(㎥/s)"].head()

In [ ]:
df["평균기온(°C)"] = df["평균기온(°C)"].astype(float)/abs(df["평균기온(°C)"].astype(float).max())
df["평균기온(°C)"].head()

In [ ]:
df

In [ ]:
plt.scatter(df["평균기온(°C)"], df["유량(㎥/s)"])
plt.xlabel("평균기온(°C)")
plt.ylabel("유량(㎥/s)")
plt.title("기온에 따른 유량 그래프")
plt.show()

In [ ]:
plt.scatter(df["유량(㎥/s)"], df["일강수량(mm)"])
plt.xlabel("일강수량(mm)")
plt.ylabel("유량(㎥/s)")
plt.title("강수량에 따른 유량 그래프")
plt.show()

In [ ]:
plt.scatter(df["평균기온(°C)"], df["일강수량(mm)"])
plt.xlabel("평균기온(°C)")
plt.ylabel("일강수량(mm)")
plt.title("기온에 따른 강수량 그래프")
plt.show()

In [ ]:
df["일강수량(mm)"] = df["일강수량(mm)"].astype(float)/abs(df["일강수량(mm)"].astype(float).max())
df["일강수량(mm)"].head()

In [ ]:
# 데이터 설정
x = df["평균기온(°C)"]
y1 = df["일강수량(mm)"]
y2 = df["유량(㎥/s)"]

# 그래프 그리기
plt.scatter(x, y1, label='일강수량')
plt.scatter(x, y2, label='유량')

# 그래프 제목, 축 레이블 설정
plt.title('기온에 따른 강수량/유량비교')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.show()
#그래프를 통해 강수량과 유량은 서로 연관성이 있음을 파악할 수 있음
#또한 그래프에서 0.8의 지점에서 강수량과 유량이 갑자기 증가하는 것을 보아 여름 장마철이라는 것을 알 수 있음

In [ ]:
# 데이터 설정
x = df["유량(㎥/s)"]
y1 = df["일강수량(mm)"]
y2 = df["평균 상대습도(%)"]

# 그래프 그리기
plt.scatter(x, y1, label='일강수량')
plt.scatter(x, y2, label='평균 상대습도')

# 그래프 제목, 축 레이블 설정
plt.title('일강수량과 평균 상대습도 비교')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.show()
#습도와 강수량은 서로 매칭되지 않음, 그래프를 통해 강수량이 적을수록 습도가 높아지는 것을 알 수 있음
#강수량과 습도는 서로 비례관계를 나타낼 것이라고 생각했지만 오산이었다....

In [ ]:
# 데이터 설정
x = df["년/월/일"]
y1 = df["평균기온(°C)"]
y2 = df["유량(㎥/s)"]

# 그래프 그리기
plt.plot(x, y1, label='평균기온')
plt.plot(x, y2, label='유량')

# 그래프 제목, 축 레이블 설정
plt.title('일강수량과 평균 상대습도 비교')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')

# 범례(legend) 표시
plt.legend()

# 그래프 보이기
plt.show()
#2011년과 2020년의 유량 증가율이 가장 높았다.

In [ ]:
#MLP를 이용해 유량을 예측해본다.
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
#강수량에 따른 유량을 예측해보는 프로그램을 생성해보았다.
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#adam
#Test loss: 0.0025083599612116814
#1/1 [==============================] - 0s 67ms/step
#Next value prediction: [[0.01919698]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Nadam', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#NADAM
#Test loss: 0.002967906417325139
#1/1 [==============================] - 0s 47ms/step
#Next value prediction: [[0.02090745]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='SGD', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#SGD
#Test loss: 0.005419999826699495
#1/1 [==============================] - 0s 39ms/step
#Next value prediction: [[0.02205948]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adagrad', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adagrad
#Test loss: 0.005659674759954214
#1/1 [==============================] - 0s 42ms/step
#Next value prediction: [[0.01848376]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='Adadelta', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#Adadelta
#Test loss: 0.008267509751021862
#1/1 [==============================] - 0s 44ms/step
#Next value prediction: [[0.03080308]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=150, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#RMSPRop
#Test loss: 0.002469925209879875
#1/1 [==============================] - 0s 46ms/step
#Next value prediction: [[0.02686775]]

In [ ]:
#표로 정리
data = {
    'Optimizer': ['adam', 'nadam', 'SGD', 'Adagrad', 'Adadelta', 'RMSPRop'],
    '오차': [0.0025083599612116814, 0.002967906417325139, 0.005419999826699495, 0.005659674759954214, 0.008267509751021862, 0.002469925209879875],
    'Epoch': [150, 150, 150, 150, 150, 150],
    'Next value prediction': [0.01919698, 0.02090745, 0.02205948, 0.01848376, 0.03080308, 0.02686775],
}

df_8 = pd.DataFrame(data, dtype=float)

In [ ]:
df_8

In [ ]:
#가장 오차가 작은 것을 알아보기 위해 그래프로 나타내보았다.
import numpy as np
import matplotlib.pyplot as plt
x = df_8["Optimizer"]
y1 = df_8["오차"]

# 그래프 그리기
plt.bar(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.ylabel('Y-axis')

# 그래프 보이기
plt.show()

#오차가 가장 큰 것은 Adadelta였고, 오차가 가장 작은 옵티마이저는 RMSProp였다.

In [ ]:
#오차가 가장 작게 나온 옵티마이저를 활용하여 batch_size, epoch등 다양한 값을 변동시켜볼 생각이다.

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=32)
#48/24
#Test loss: 0.002667686901986599
#1/1 [==============================] - 0s 63ms/step
#Next value prediction: [[0.01267303]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=32)
#96/48
#Test loss: 0.002863157307729125
#1/1 [==============================] - 0s 49ms/step
#Next value prediction: [[0.01077946]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))
print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=64)
#96/48
#Test loss: 0.002391877118498087
#1/1 [==============================] - 0s 43ms/step
#Next value prediction: [[0.01014327]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))

print('Next value prediction:', next_value)

In [ ]:
#epochs=200, batch_size=64)
#48/24
#Test loss: 0.002675211988389492
#1/1 [==============================] - 0s 45ms/step
#Next value prediction: [[0.01967538]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))

print('Next value prediction:', next_value)

In [ ]:
#epochs=500, batch_size=64)
#96/48
#Test loss: 0.0030212218407541513
#1/1 [==============================] - 0s 47ms/step
#Next value prediction: [[0.01965078]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#96/48
#epochs=500, batch_size=32)
#Test loss: 0.0034068876411765814
#1/1 [==============================] - 0s 41ms/step
#Next value prediction: [[0.03192162]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#Test loss: 0.0026609799824655056
#1/1 [==============================] - 0s 47ms/step
#Next value prediction: [[0.02860419]]
#48/24
#epochs=500, batch_size=64)

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=500, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#epochs=500, batch_size=64)
#96/48
#Test loss: 0.003773909527808428
#1/1 [==============================] - 0s 46ms/step
#Next value prediction: [[0.00917505]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#epochs=1000, batch_size=64)
#96/48
#Test loss: 0.0035504396073520184
#1/1 [==============================] - 0s 41ms/step
#Next value prediction: [[0.01316074]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(96, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(48, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#Test loss: 0.0035440537612885237
#1/1 [==============================] - 0s 47ms/step
#epochs=1000, batch_size=32)
#96/48
#Next value prediction: [[0.01608789]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=32)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#epochs=1000, batch_size=32)
#48/24
#Test loss: 0.0038421659264713526
#1/1 [==============================] - 0s 39ms/step
#Next value prediction: [[0.01999253]]

In [ ]:
data = df["일강수량(mm)"]
target = df["유량(㎥/s)"]

In [ ]:
df_1 = pd.DataFrame({'data': data, 'target': target})

In [ ]:
# 데이터 전처리
import numpy as np
window_size = 10  # 입력 시퀀스의 길이
x = []
y = []
for i in range(len(df) - window_size):
    x.append(df_1['data'].iloc[i:i+window_size].values)
    y.append(df_1['target'].iloc[i+window_size])
x = np.array(x)
y = np.array(y)

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
train_size = int(len(x) * 0.8)
x_train, x_test = x[:train_size], x[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# MLP 모델 구축
model = keras.Sequential([
    keras.layers.Dense(48, activation='relu', input_shape=(window_size,)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1)
])

# 모델 컴파일
model.compile(optimizer='RMSProp', loss='mean_squared_error')

In [ ]:
model.fit(x_train, y_train, epochs=1000, batch_size=64)

# 모델 평가
test_loss = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)

# 다음 값을 예측
last_window = df_1['data'].iloc[-window_size:].values
next_value = model.predict(np.expand_dims(last_window, axis=0))


print('Next value prediction:', next_value)

In [ ]:
#epochs=1000, batch_size=64)
#48/24
#Test loss: 0.0050267912447452545
#1/1 [==============================] - 0s 53ms/step
#Next value prediction: [[0.02314872]]

In [ ]:
#표로 정리
data = {
    "번호": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'Keras의 Dense 층': ['48/24', '96/48', '96/48', '48/24', '96/48', '96/48', '48/24', '48/24', '96/48', '96/48', '48/24', '48/24'],
    '오차': [0.002667686901986599, 0.002863157307729125, 0.002391877118498087, 0.002675211988389492, 0.0030212218407541513, 0.0034068876411765814, 0.0026609799824655056, 0.003773909527808428, 0.0035504396073520184, 0.0035440537612885237, 0.0038421659264713526, 0.0050267912447452545],
    'Epoch': [200, 200, 200, 200, 500, 500, 500, 500, 1000, 1000, 1000, 1000],
    'Next value prediction': [0.01267303, 0.01077946, 0.01014327, 0.01967538, 0.01965078, 0.03192162, 0.02860419, 0.00917505, 0.01316074, 0.01608789, 0.01999253, 0.02314872],
    'Batch_size': [32, 32, 64, 64, 64, 32, 64, 64, 64, 32, 32, 64]
}

df_9 = pd.DataFrame(data, dtype=float)

In [ ]:
df_9

In [ ]:
#그래프를 통해 오차를 파악하겠다.
#가장 오차가 작은 것을 알아보기 위해 그래프로 나타내보았다.
import numpy as np
import matplotlib.pyplot as plt
x = df_9["번호"]
y1 = df_9["오차"]

# 그래프 그리기
plt.plot(x, y1, label='error')

# 그래프 제목, 축 레이블 설정
plt.title('What would be the most appropriate?')
plt.ylabel('Y-axis')

# 그래프 보이기
plt.show()

#오차가 가장 작은 모델은 3번	keras의 Dense층 : 96/48, epoch:	200.0, batch	64.0